In [1]:
import os

os.chdir('/Users/shreygupta/Documents/Classes/CS598DLH/')
os.getcwd()

'/Users/shreygupta/Documents/Classes/CS598DLH'

In [2]:
import numpy as np
import csv
import pandas as pd
import collections
from feature_generation import FeatureGeneration
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import ExtraTreesClassifier
import weka.core.jvm as jvm
from weka.core.converters import Loader
from weka.filters import Filter
from weka.attribute_selection import ASEvaluation, AttributeSelection
from weka.classifiers import Classifier, Evaluation
from imblearn.over_sampling import SMOTE

jvm.start()


DEBUG:weka.core.jvm:Adding bundled jars
DEBUG:weka.core.jvm:Classpath=['/Users/shreygupta/Documents/Classes/CS598DLH/.conda/lib/python3.10/site-packages/javabridge/jars/rhino-1.7R4.jar', '/Users/shreygupta/Documents/Classes/CS598DLH/.conda/lib/python3.10/site-packages/javabridge/jars/runnablequeue.jar', '/Users/shreygupta/Documents/Classes/CS598DLH/.conda/lib/python3.10/site-packages/javabridge/jars/cpython.jar', '/Users/shreygupta/Documents/Classes/CS598DLH/.conda/lib/python3.10/site-packages/weka/lib/python-weka-wrapper.jar', '/Users/shreygupta/Documents/Classes/CS598DLH/.conda/lib/python3.10/site-packages/weka/lib/weka.jar']
DEBUG:weka.core.jvm:MaxHeapSize=default
DEBUG:weka.core.jvm:Package support disabled
Traceback (most recent call last):
  File "/Users/shreygupta/Documents/Classes/CS598DLH/.conda/lib/python3.10/site-packages/javabridge/jutil.py", line 282, in start_thread
    vm.create_mac(args, RQCLS, library_path, libjli_path)
  File "_javabridge.pyx", line 709, in _javabridg

Failed to open libjli.dylib.



RuntimeError: Failed to start Java VM

In [39]:
class DecisionTree:
    def __init__(self, x_train, y_train, x_test, y_test, k):
        self.dtc = DecisionTreeClassifier(splitter='random', random_state=42)
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        self.k = k
        
    def feature_selection_SelectKBest(self):
        k_best = SelectKBest(chi2, k=self.k)
        k_best.fit(self.x_train, self.y_train)
        self.x_train = k_best.transform(self.x_train)
        self.x_test = k_best.transform(self.x_test)
        
    def feature_selection_ExtraTreesClassifier(self):
        clf = ExtraTreesClassifier(n_estimators=100, random_state=42)
        clf.fit(self.x_train, self.y_train)
        importances = clf.feature_importances_
        indices = np.argsort(importances)[::-1]
        self.x_train = self.x_train[:, indices[:self.k]]
        self.x_test = self.x_test[:, indices[:self.k]]

    def feature_selection_InfoGainAttributeEval(self, morbidity):
        loader = Loader(classname="weka.core.converters.ArffLoader")
        train_data = loader.load_file(f"./dataset/train/train_{morbidity}_tfidf.arff")
        train_data.class_is_last()

        # Initialize attribute selection
        eval = ASEvaluation(classname="weka.attributeSelection.InfoGainAttributeEval")
        search = AttributeSelection()
        search.evaluator = eval
        search.select_attributes(train_data)
        selected_attributes = search.selected_attributes
        filtered_attributes = np.delete(selected_attributes, [-1])

        # Apply selected attributes to the training and testing sets
        self.x_train = self.x_train[:, filtered_attributes]
        self.x_test = self.x_test[:, filtered_attributes]

    def train(self):
        self.dtc.fit(self.x_train, self.y_train)

    def test_and_evaluate(self):
        y_pred = self.dtc.predict(self.x_test)
        f1_macro = f1_score(self.y_test, y_pred, average='macro')
        f1_micro = f1_score(self.y_test, y_pred, average='micro')
        return f1_macro, f1_micro

In [40]:
morbidities = ['Asthma', 'CAD', 'CHF', 'Depression', 'Diabetes', 'Gallstones', 'GERD', 'Gout', 'Hypercholesterolemia', 'Hypertension', 'Hypertriglyceridemia', 'OA', 'Obesity', 'OSA', 'PVD', 'Venous_Insufficiency']

column_headings = ["Morbidity Class", "DT_Macro F1", "DT_Micro F1"]
    

### All Features

In [41]:
with open("./results/tf-idf/performance_DT_AllFeatures.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2]])
    
all_f1_macro_scores = []
all_f1_micro_scores = []

for morbidity in morbidities:
    train_preprocessed_df = pd.read_csv('./dataset/train/train_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = FeatureGeneration(train_preprocessed_df, morbidity).tf_idf()

    if len(collections.Counter(list(Y)).keys()) >=2:
        smote = SMOTE(random_state=42,k_neighbors=2)
        X_train_resampled, y_train_resampled = smote.fit_resample(X, Y)
        X, Y =  X_train_resampled, y_train_resampled

        # add KFold cross validation
        skf = KFold(n_splits=10, shuffle=True, random_state=42)

        f1_macro_list = []
        f1_micro_list = []
        for train_idx, val_idx in skf.split(X, Y):
            X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
            X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

            # Training RF using TF-IDF Representation
            dt_obj = DecisionTree(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 0)
            dt_obj.train()

            f1_macro, f1_micro = dt_obj.test_and_evaluate()

            f1_macro_list.append(f1_macro)
            f1_micro_list.append(f1_micro)

        f1_macro = np.mean(f1_macro_list)
        f1_micro = np.mean(f1_micro_list)
    else:
        f1_macro = 1
        f1_micro = 1
    
    print(f"{morbidity} == Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]
    all_f1_macro_scores.append(f1_macro)
    all_f1_micro_scores.append(f1_micro)

    with open("./results/tf-idf/performance_DT_AllFeatures.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)


with open("./results/tf-idf/performance_DT_AllFeatures.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Average"]
    row.extend([sum(all_f1_macro_scores)/len(all_f1_macro_scores),  sum(all_f1_micro_scores)/len(all_f1_micro_scores) ])
    writer.writerow(row)

(572, 600) (572,) Counter({0.0: 502, 1.0: 70})
Asthma == Macro F1 score: 0.9546519505256145 and Micro F1 Score 0.9551683168316831
(548, 600) (548,) Counter({1.0: 325, 0.0: 223})
CAD == Macro F1 score: 0.8536166138521624 and Micro F1 Score 0.8553846153846155
(243, 600) (243,) Counter({1.0: 243})
CHF == Macro F1 score: 1 and Micro F1 Score 1
(582, 600) (582,) Counter({0.0: 460, 1.0: 122})
Depression == Macro F1 score: 0.8789546355205691 and Micro F1 Score 0.8804347826086957
(567, 600) (567,) Counter({1.0: 396, 0.0: 171})
Diabetes == Macro F1 score: 0.9415394268987483 and Micro F1 Score 0.9418987341772151
(593, 600) (593,) Counter({0.0: 506, 1.0: 87})
Gallstones == Macro F1 score: 0.8418077579457431 and Micro F1 Score 0.8438167346146379
(487, 600) (487,) Counter({0.0: 372, 1.0: 115})
GERD == Macro F1 score: 0.7426973329922422 and Micro F1 Score 0.7459639639639639
(596, 600) (596,) Counter({0.0: 518, 1.0: 78})
Gout == Macro F1 score: 0.8619235790717091 and Micro F1 Score 0.8639563106796118

### Select K Best k=100

In [42]:
with open("./results/tf-idf/performance_DT_SelectKBest.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2]])
    
all_f1_macro_scores = []
all_f1_micro_scores = []

for morbidity in morbidities:
    train_preprocessed_df = pd.read_csv('./dataset/train/train_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = FeatureGeneration(train_preprocessed_df, morbidity).tf_idf()

    if len(collections.Counter(list(Y)).keys()) >=2:
        smote = SMOTE(random_state=42,k_neighbors=2)
        X_train_resampled, y_train_resampled = smote.fit_resample(X, Y)
        X, Y =  X_train_resampled, y_train_resampled
        
        # add KFold cross validation
        skf = KFold(n_splits=10, shuffle=True, random_state=42)

        f1_macro_list = []
        f1_micro_list = []
        for train_idx, val_idx in skf.split(X, Y):
            X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
            X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

            # Training RF using TF-IDF Representation
            dt_obj = DecisionTree(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 100)
            dt_obj.feature_selection_SelectKBest()
            dt_obj.train()   

            f1_macro, f1_micro = dt_obj.test_and_evaluate()

            f1_macro_list.append(f1_macro)
            f1_micro_list.append(f1_micro)

        f1_macro = np.mean(f1_macro_list)
        f1_micro = np.mean(f1_micro_list)
    else:
        f1_macro = 1
        f1_micro = 1
    
    print(f"{morbidity} == Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]
    all_f1_macro_scores.append(f1_macro)
    all_f1_micro_scores.append(f1_micro)

    with open("./results/tf-idf/performance_DT_SelectKBest.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)


with open("./results/tf-idf/performance_DT_SelectKBest.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([sum(all_f1_macro_scores)/len(all_f1_macro_scores),  sum(all_f1_micro_scores)/len(all_f1_micro_scores) ])
    writer.writerow(row)

(572, 600) (572,) Counter({0.0: 502, 1.0: 70})
Asthma == Macro F1 score: 0.9597104872709916 and Micro F1 Score 0.9601584158415843
(548, 600) (548,) Counter({1.0: 325, 0.0: 223})
CAD == Macro F1 score: 0.8757710466742681 and Micro F1 Score 0.876923076923077
(243, 600) (243,) Counter({1.0: 243})
CHF == Macro F1 score: 1 and Micro F1 Score 1
(582, 600) (582,) Counter({0.0: 460, 1.0: 122})
Depression == Macro F1 score: 0.8857780990267348 and Micro F1 Score 0.8869565217391304
(567, 600) (567,) Counter({1.0: 396, 0.0: 171})
Diabetes == Macro F1 score: 0.9442102840154296 and Micro F1 Score 0.9445727848101265
(593, 600) (593,) Counter({0.0: 506, 1.0: 87})
Gallstones == Macro F1 score: 0.8453967927481836 and Micro F1 Score 0.8478353717724714
(487, 600) (487,) Counter({0.0: 372, 1.0: 115})
GERD == Macro F1 score: 0.7575748844168186 and Micro F1 Score 0.7635675675675676
(596, 600) (596,) Counter({0.0: 518, 1.0: 78})
Gout == Macro F1 score: 0.875802855748414 and Micro F1 Score 0.8773898431665422
(

### ExtraTreesClassifier

In [43]:
with open("./results/tf-idf/performance_DT_ExtraTreesClassifier.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2]])
    
all_f1_macro_scores = []
all_f1_micro_scores = []

for morbidity in morbidities:

    train_preprocessed_df = pd.read_csv('./dataset/train/train_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = FeatureGeneration(train_preprocessed_df, morbidity).tf_idf()

    if len(collections.Counter(list(Y)).keys()) >=2:
        smote = SMOTE(random_state=42,k_neighbors=2)
        X_train_resampled, y_train_resampled = smote.fit_resample(X, Y)
        X, Y =  X_train_resampled, y_train_resampled
        
        # add KFold cross validation
        skf = KFold(n_splits=10, shuffle=True, random_state=42)

        f1_macro_list = []
        f1_micro_list = []
        for train_idx, val_idx in skf.split(X, Y):
            X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
            X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

            # Training RF using TF-IDF Representation
            dt_obj = DecisionTree(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 100)
            dt_obj.feature_selection_ExtraTreesClassifier()
            dt_obj.train()

            f1_macro, f1_micro = dt_obj.test_and_evaluate()

            f1_macro_list.append(f1_macro)
            f1_micro_list.append(f1_micro)

        f1_macro = np.mean(f1_macro_list)
        f1_micro = np.mean(f1_micro_list)
    else:
        f1_macro = 1
        f1_micro = 1
    
    print(f"{morbidity} == Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]
    all_f1_macro_scores.append(f1_macro)
    all_f1_micro_scores.append(f1_micro)

    with open("./results/tf-idf/performance_DT_ExtraTreesClassifier.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)


with open("./results/tf-idf/performance_DT_ExtraTreesClassifier.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([sum(all_f1_macro_scores)/len(all_f1_macro_scores),  sum(all_f1_micro_scores)/len(all_f1_micro_scores) ])
    writer.writerow(row)

(572, 600) (572,) Counter({0.0: 502, 1.0: 70})
Asthma == Macro F1 score: 0.9626619377382826 and Micro F1 Score 0.9631386138613861
(548, 600) (548,) Counter({1.0: 325, 0.0: 223})
CAD == Macro F1 score: 0.8571311193183095 and Micro F1 Score 0.8584615384615384
(243, 600) (243,) Counter({1.0: 243})
CHF == Macro F1 score: 1 and Micro F1 Score 1
(582, 600) (582,) Counter({0.0: 460, 1.0: 122})
Depression == Macro F1 score: 0.8489901794922758 and Micro F1 Score 0.851086956521739
(567, 600) (567,) Counter({1.0: 396, 0.0: 171})
Diabetes == Macro F1 score: 0.930195821551685 and Micro F1 Score 0.9305379746835442
(593, 600) (593,) Counter({0.0: 506, 1.0: 87})
Gallstones == Macro F1 score: 0.8472939805522086 and Micro F1 Score 0.8508056688021742
(487, 600) (487,) Counter({0.0: 372, 1.0: 115})
GERD == Macro F1 score: 0.7579833350518224 and Micro F1 Score 0.7595855855855855
(596, 600) (596,) Counter({0.0: 518, 1.0: 78})
Gout == Macro F1 score: 0.8689602147110123 and Micro F1 Score 0.870687079910381
(5

### Info Gain

In [44]:
with open("./results/tf-idf/performance_DT_InfoGain.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2]])
    
all_f1_macro_scores = []
all_f1_micro_scores = []

for morbidity in morbidities:
    train_preprocessed_df = pd.read_csv('./dataset/train/train_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = FeatureGeneration(train_preprocessed_df, morbidity).tf_idf()

    
    if len(collections.Counter(list(Y)).keys()) < 2:
        f1_macro = 1
        f1_micro = 1
    else:
        smote = SMOTE(random_state=42,k_neighbors=2)
        X_train_resampled, y_train_resampled = smote.fit_resample(X, Y)
        X, Y =  X_train_resampled, y_train_resampled
        
        # add KFold cross validation
        skf = KFold(n_splits=10, shuffle=True, random_state=42)

        f1_macro_list = []
        f1_micro_list = []
        for train_idx, val_idx in skf.split(X, Y):
            X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
            X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

            # Training RF using TF-IDF Representation
            dt_obj = DecisionTree(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 100)
            dt_obj.feature_selection_InfoGainAttributeEval(morbidity=morbidity)
            # dt_obj.feature_selection_InfoGainAttributeEval(morbidity)
            dt_obj.train()

            f1_macro, f1_micro = dt_obj.test_and_evaluate()

            f1_macro_list.append(f1_macro)
            f1_micro_list.append(f1_micro)

        f1_macro = np.mean(f1_macro_list)
        f1_micro = np.mean(f1_micro_list)
    print(f"{morbidity} == Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]
    all_f1_macro_scores.append(f1_macro)
    all_f1_micro_scores.append(f1_micro)

    with open("./results/tf-idf/performance_DT_InfoGain.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)


with open("./results/tf-idf/performance_DT_InfoGain.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([sum(all_f1_macro_scores)/len(all_f1_macro_scores),  sum(all_f1_micro_scores)/len(all_f1_micro_scores) ])
    writer.writerow(row)

(572, 600) (572,) Counter({0.0: 502, 1.0: 70})


AttributeError: 'NoneType' object has no attribute 'find_class'